<a href="https://colab.research.google.com/github/yajima-yasutoshi/Model/blob/main/20250723/%E6%96%BD%E8%A8%AD%E9%85%8D%E7%BD%AE%E5%95%8F%E9%A1%8C%E3%81%AE%E6%BC%94%E7%BF%92%E5%95%8F%E9%A1%8C%E3%81%AE%E8%A7%A3%E8%AA%AC%E3%81%A8%E8%A7%A3%E7%AD%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#施設配置問題の演習問題
まず、問題を解くために必要なライブラリをインストールする。

In [ ]:
%%capture
!pip install mip

## 演習問題1

例題において、固定費用が以下のように変更された場合、最適な施設配置における総コストを求めよ。

  * F1: 固定費用 10
  * F2: 固定費用 12
  * F3: 固定費用 8

**集合とパラメータ:**

  * $I$: 施設候補地の集合 $\{F1, F2, F3\}$
  * $J$: 顧客の集合 $\{C1, C2, C3, C4\}$
  * $f_i$: 施設 $i \in I$ を開設するための固定費用 ( **変更点** )
      * $f\_{F1} = 10$
      * $f\_{F2} = 12$
      * $f\_{F3} = 8$
  * $c_{ij}$: 施設 $i \in I$ から顧客 $j \in J$ へサービスを提供するための輸送費用

**決定変数:**

  * $y_i$: 施設 $i \in I$ を開設するなら1、しないなら0
  * $x_{ij}$: 顧客 $j \in J$ を施設 $i \in I$ に割り当てるなら1、しないなら0

**目的関数:**
$$\text{Minimize} \quad Z = \sum_{i \in I} f_i y_i + \sum_{i \in I} \sum_{j \in J} c_{ij} x_{ij}$$

**制約条件:**

1.  各顧客は、必ずいずれか1つの施設に割り当てられる。
    $$\sum_{i \in I} x_{ij} = 1 \quad \forall j \in J$$
2.  顧客は、開設された施設からのみサービスを受けられる。
    $$x_{ij} \le y_i \quad \forall i \in I, \forall j \in J$$
3.  変数の型はバイナリ変数である。
$$y_i \in \{0, 1\} \quad \forall i \in I$$
$$x_{ij} \in \{0, 1\} \quad \forall i \in I, \forall j \in J$$

###Python (MIP) による実装

In [ ]:
from mip import Model, xsum, minimize, BINARY, OptimizationStatus

# --- データ定義 ---
# 施設候補地
facilities = ['F1', 'F2', 'F3']
# 演習問題1の固定費用
fixed_costs = {'F1': 10, 'F2': 12, 'F3': 8}

# 顧客
customers = ['C1', 'C2', 'C3', 'C4']

# 輸送費用 (facility, customer) -> cost
transport_costs = {
    ('F1', 'C1'): 20, ('F1', 'C2'): 10, ('F1', 'C3'): 50, ('F1', 'C4'): 30,
    ('F2', 'C1'): 60, ('F2', 'C2'): 30, ('F2', 'C3'): 20, ('F2', 'C4'): 10,
    ('F3', 'C1'): 10, ('F3', 'C2'): 40, ('F3', 'C3'): 15, ('F3', 'C4'): 50,
}

# --- モデル作成 ---
model = Model("FacilityLocation_Ex1")

# --- 決定変数 ---
# y_i: 施設iを開設するかどうか (バイナリ変数)
y = {i: model.add_var(var_type=BINARY, name=f"y_{i}") for i in facilities}
# x_ij: 顧客jを施設iに割り当てるかどうか (バイナリ変数)
x = {(i, j): model.add_var(var_type=BINARY, name=f"x_{i}_{j}") for i in facilities for j in customers}

# --- 目的関数 ---
# 総費用 = 固定費用 + 輸送費用
model.objective = minimize(
    xsum(fixed_costs[i] * y[i] for i in facilities) +
    xsum(transport_costs[i, j] * x[i, j] for i in facilities for j in customers)
)

# --- 制約条件 ---
# 1. 各顧客は必ず1つの施設に割り当てられる
for j in customers:
    model += xsum(x[i, j] for i in facilities) == 1, f"AssignCust_{j}"

# 2. 顧客はその施設が開設されている場合にのみ割り当て可能
for i in facilities:
    for j in customers:
        model += x[i, j] <= y[i], f"OpenFac_{i}_{j}"

# --- モデルの最適化 ---
status = model.optimize()

# --- 結果の表示 ---
print("--- 演習問題1 結果 ---")
if status == OptimizationStatus.OPTIMAL:
    print(f"最適解が見つかりました。")
    print(f"最適な総コスト: {model.objective_value:.2f}")

    print("\n開設された施設:")
    for i in facilities:
        if y[i].x >= 0.99:
            print(f"- {i}")

    print("\n顧客の割り当て:")
    for j in customers:
        for i in facilities:
            if x[i, j].x >= 0.99:
                print(f"- 顧客 {j} -> 施設 {i} (輸送費用: {transport_costs[i, j]})")
elif status == OptimizationStatus.INFEASIBLE:
    print("実行不可能な問題です。制約を満たす解が存在しません。")
else:
    print(f"最適化が停止しました。ステータス: {status}")

--- 演習問題1 結果 ---
最適解が見つかりました。
最適な総コスト: 75.00

開設された施設:
- F1
- F2
- F3

顧客の割り当て:
- 顧客 C1 -> 施設 F3 (輸送費用: 10)
- 顧客 C2 -> 施設 F1 (輸送費用: 10)
- 顧客 C3 -> 施設 F3 (輸送費用: 15)
- 顧客 C4 -> 施設 F2 (輸送費用: 10)


-----

## 演習問題2

例題において、建設できる施設の最大数を1箇所に制限する場合、最適な施設配置を求め、総コストを解答せよ。

###数理モデルの定式化

この問題では、開設される施設の総数を1以下に制限する制約が追加される。

**追加される制約条件:**

  * 開設される施設の総数は1以下でなければならない。
$$\sum_{i \in I} y_i \le 1$$

その他の決定変数、目的関数、制約条件は例題のモデルと同一である。

###Python (MIP) による実装

In [ ]:
from mip import Model, xsum, minimize, BINARY, OptimizationStatus

# --- データ定義 ---
# 施設候補地
facilities = ['F1', 'F2', 'F3']
# 例題の固定費用
fixed_costs = {'F1': 100, 'F2': 120, 'F3': 80}

# 顧客
customers = ['C1', 'C2', 'C3', 'C4']

# 輸送費用 (facility, customer) -> cost
transport_costs = {
    ('F1', 'C1'): 20, ('F1', 'C2'): 10, ('F1', 'C3'): 50, ('F1', 'C4'): 30,
    ('F2', 'C1'): 60, ('F2', 'C2'): 30, ('F2', 'C3'): 20, ('F2', 'C4'): 10,
    ('F3', 'C1'): 10, ('F3', 'C2'): 40, ('F3', 'C3'): 15, ('F3', 'C4'): 50,
}

# --- モデル作成 ---
model = Model("FacilityLocation_Ex2")

# --- 決定変数 ---
y = {i: model.add_var(var_type=BINARY, name=f"y_{i}") for i in facilities}
x = {(i, j): model.add_var(var_type=BINARY, name=f"x_{i}_{j}") for i in facilities for j in customers}

# --- 目的関数 ---
model.objective = minimize(
    xsum(fixed_costs[i] * y[i] for i in facilities) +
    xsum(transport_costs[i, j] * x[i, j] for i in facilities for j in customers)
)

# --- 制約条件 ---
# 1. 各顧客は必ず1つの施設に割り当てられる
for j in customers:
    model += xsum(x[i, j] for i in facilities) == 1, f"AssignCust_{j}"

# 2. 顧客はその施設が開設されている場合にのみ割り当て可能
for i in facilities:
    for j in customers:
        model += x[i, j] <= y[i], f"OpenFac_{i}_{j}"

# 3. 開設できる施設の最大数は1つ (演習問題2の追加制約)
model += xsum(y[i] for i in facilities) <= 1, "MaxOneFacility"

# --- モデルの最適化 ---
status = model.optimize()

# --- 結果の表示 ---
print("--- 演習問題2 結果 ---")
if status == OptimizationStatus.OPTIMAL:
    print(f"最適解が見つかりました。")
    print(f"最適な総コスト: {model.objective_value:.2f}")

    print("\n開設された施設:")
    for i in facilities:
        if y[i].x >= 0.99:
            print(f"- {i}")

    print("\n顧客の割り当て:")
    for j in customers:
        for i in facilities:
            if x[i, j].x >= 0.99:
                print(f"- 顧客 {j} -> 施設 {i} (輸送費用: {transport_costs[i, j]})")
else:
    print(f"最適化が停止しました。ステータス: {status}")

--- 演習問題2 結果 ---
最適解が見つかりました。
最適な総コスト: 195.00

開設された施設:
- F3

顧客の割り当て:
- 顧客 C1 -> 施設 F3 (輸送費用: 10)
- 顧客 C2 -> 施設 F3 (輸送費用: 40)
- 顧客 C3 -> 施設 F3 (輸送費用: 15)
- 顧客 C4 -> 施設 F3 (輸送費用: 50)


-----

## 演習問題3

例題において、施設F1は必ず建設しなければならない、という条件が加わった場合、最適な施設配置と総コストはどうなるか。

###数理モデルの定式化

この問題では、特定の施設（F1）の開設を強制する制約をモデルに追加する。これは、施設F1の開設を示すバイナリ変数 $y\_{F1}$ を1に固定することで表現できる。

**追加される制約条件:**

  * 施設F1は必ず開設されなければならない。
$$y_{F1} = 1$$

その他の決定変数、目的関数、制約条件は例題のモデルと同一である。

###Python (MIP) による実装

In [ ]:
from mip import Model, xsum, minimize, BINARY, OptimizationStatus

# --- データ定義 ---
# 施設候補地
facilities = ['F1', 'F2', 'F3']
# 例題の固定費用
fixed_costs = {'F1': 100, 'F2': 120, 'F3': 80}

# 顧客
customers = ['C1', 'C2', 'C3', 'C4']

# 輸送費用 (facility, customer) -> cost
transport_costs = {
    ('F1', 'C1'): 20, ('F1', 'C2'): 10, ('F1', 'C3'): 50, ('F1', 'C4'): 30,
    ('F2', 'C1'): 60, ('F2', 'C2'): 30, ('F2', 'C3'): 20, ('F2', 'C4'): 10,
    ('F3', 'C1'): 10, ('F3', 'C2'): 40, ('F3', 'C3'): 15, ('F3', 'C4'): 50,
}

# --- モデル作成 ---
model = Model("FacilityLocation_Ex3")

# --- 決定変数 ---
y = {i: model.add_var(var_type=BINARY, name=f"y_{i}") for i in facilities}
x = {(i, j): model.add_var(var_type=BINARY, name=f"x_{i}_{j}") for i in facilities for j in customers}

# --- 目的関数 ---
model.objective = minimize(
    xsum(fixed_costs[i] * y[i] for i in facilities) +
    xsum(transport_costs[i, j] * x[i, j] for i in facilities for j in customers)
)

# --- 制約条件 ---
# 1. 各顧客は必ず1つの施設に割り当てられる
for j in customers:
    model += xsum(x[i, j] for i in facilities) == 1, f"AssignCust_{j}"

# 2. 顧客はその施設が開設されている場合にのみ割り当て可能
for i in facilities:
    for j in customers:
        model += x[i, j] <= y[i], f"OpenFac_{i}_{j}"

# 3. 施設F1は必ず建設する (演習問題3の追加制約)
model += y['F1'] == 1, "ForceOpen_F1"

# --- モデルの最適化 ---
status = model.optimize()

# --- 結果の表示 ---
print("--- 演習問題3 結果 ---")
if status == OptimizationStatus.OPTIMAL:
    print(f"最適解が見つかりました。")
    print(f"最適な総コスト: {model.objective_value:.2f}")

    print("\n開設された施設:")
    for i in facilities:
        if y[i].x >= 0.99:
            print(f"- {i}")

    print("\n顧客の割り当て:")
    for j in customers:
        for i in facilities:
            if x[i, j].x >= 0.99:
                print(f"- 顧客 {j} -> 施設 {i} (輸送費用: {transport_costs[i, j]})")
else:
    print(f"最適化が停止しました。ステータス: {status}")

--- 演習問題3 結果 ---
最適解が見つかりました。
最適な総コスト: 210.00

開設された施設:
- F1

顧客の割り当て:
- 顧客 C1 -> 施設 F1 (輸送費用: 20)
- 顧客 C2 -> 施設 F1 (輸送費用: 10)
- 顧客 C3 -> 施設 F1 (輸送費用: 50)
- 顧客 C4 -> 施設 F1 (輸送費用: 30)


-----

## 演習問題4

例題に以下の条件を追加する。

  * 各施設が対応できる最大の顧客数 $K_i$ は以下の通りとする。
      * $K\_{F1} = 3$
      * $K\_{F2} = 2$
      * $K\_{F3} = 2$

この条件の下で、最適な総コストを求めよ。

###数理モデルでの定式化

この問題は、施設の容量制約を考慮した**容量制約付き施設配置問題 (Capacitated Facility Location Problem - CFLP)** である。例題のモデルに、各施設が受け持つことができる顧客数に上限を設ける制約を追加する。

**追加される制約条件:**

  * 各施設 $i$ に割り当てられる顧客の総数は、その施設の容量 $K\_i$ を超えてはならない。
$$\sum_{j \in J} x_{ij} \le K_i \cdot y_i \quad \forall i \in I$$
  この制約式では、$y_i$ を右辺に掛けることが重要である。
  もし施設 $i$ が開設されていなければ ($y_i=0$)右辺が0となり、
  その施設への顧客の割り当て ($\sum_j x_{ij}$) は強制的に0となる。
  施設が開設されていれば ($y_i=1$)割り当て数が容量 $K_i$ を超えないという制約が有効になる。

###Python (MIP) による実装

In [ ]:
from mip import Model, xsum, minimize, BINARY, OptimizationStatus

# --- データ定義 ---
# 施設候補地
facilities = ['F1', 'F2', 'F3']
# 例題の固定費用
fixed_costs = {'F1': 100, 'F2': 120, 'F3': 80}

# 顧客
customers = ['C1', 'C2', 'C3', 'C4']

# 輸送費用
transport_costs = {
    ('F1', 'C1'): 20, ('F1', 'C2'): 10, ('F1', 'C3'): 50, ('F1', 'C4'): 30,
    ('F2', 'C1'): 60, ('F2', 'C2'): 30, ('F2', 'C3'): 20, ('F2', 'C4'): 10,
    ('F3', 'C1'): 10, ('F3', 'C2'): 40, ('F3', 'C3'): 15, ('F3', 'C4'): 50,
}

# 施設の容量 (演習問題4の追加データ)
capacities = {'F1': 3, 'F2': 2, 'F3': 2}

# --- モデル作成 ---
model = Model("FacilityLocation_Ex4_Capacitated")

# --- 決定変数 ---
y = {i: model.add_var(var_type=BINARY, name=f"y_{i}") for i in facilities}
x = {(i, j): model.add_var(var_type=BINARY, name=f"x_{i}_{j}") for i in facilities for j in customers}

# --- 目的関数 ---
model.objective = minimize(
    xsum(fixed_costs[i] * y[i] for i in facilities) +
    xsum(transport_costs[i, j] * x[i, j] for i in facilities for j in customers)
)

# --- 制約条件 ---
# 1. 各顧客は必ず1つの施設に割り当てられる
for j in customers:
    model += xsum(x[i, j] for i in facilities) == 1, f"AssignCust_{j}"

# 2. 顧客はその施設が開設されている場合にのみ割り当て可能
for i in facilities:
    for j in customers:
        model += x[i, j] <= y[i], f"OpenFac_{i}_{j}"

# 3. 各施設の容量制約 (演習問題4の追加制約)
for i in facilities:
    model += xsum(x[i, j] for j in customers) <= capacities[i] * y[i], f"Capacity_{i}"

# --- モデルの最適化 ---
status = model.optimize()

# --- 結果の表示 ---
print("--- 演習問題4 結果 ---")
if status == OptimizationStatus.OPTIMAL:
    print(f"最適解が見つかりました。")
    print(f"最適な総コスト: {model.objective_value:.2f}")

    print("\n開設された施設:")
    for i in facilities:
        if y[i].x >= 0.99:
            print(f"- {i}")

    print("\n顧客の割り当て:")
    for j in customers:
        for i in facilities:
            if x[i, j].x >= 0.99:
                print(f"- 顧客 {j} -> 施設 {i} (輸送費用: {transport_costs[i, j]})")
else:
    print(f"最適化が停止しました。ステータス: {status}")

--- 演習問題4 結果 ---
最適解が見つかりました。
最適な総コスト: 245.00

開設された施設:
- F1
- F3

顧客の割り当て:
- 顧客 C1 -> 施設 F3 (輸送費用: 10)
- 顧客 C2 -> 施設 F1 (輸送費用: 10)
- 顧客 C3 -> 施設 F3 (輸送費用: 15)
- 顧客 C4 -> 施設 F1 (輸送費用: 30)


-----

## 演習問題5

例題に、新しい施設建設候補地 F4を追加する。

  * 候補地 F4 の固定費 $f_{F4} = 90$
  * 候補地 F4 から各顧客への輸送コスト $c_{F4,j} = [18, 12, 20, 10]$
    
この新しい候補地を含めて最適化した場合、総コストはいくつか。

###数理モデルでの定式化

この問題は、施設候補地の集合 $I$ を拡張して解を求めるものである。数理モデルの構造は例題のUFLPと全く同じであるが、扱う集合とパラメータのサイズが変化する。

**変更点:**

  * 施設候補地の集合: $I = \{F1, F2, F3, F4\}$
  * 追加の固定費用: $f_{F4} = 90$
  * 追加の輸送費用: $c_{F4,C1}=18, c_{F4,C2}=12, c_{F4,C3}=20, c_{F4,C4}=10$

モデルの目的関数と制約条件の数式表現は例題と同一のままである。

###Python (MIP) による実装

In [ ]:
from mip import Model, xsum, minimize, BINARY, OptimizationStatus

# --- データ定義 ---
# 施設候補地 (F4を追加)
facilities = ['F1', 'F2', 'F3', 'F4']
# 固定費用 (F4を追加)
fixed_costs = {'F1': 100, 'F2': 120, 'F3': 80, 'F4': 90}

# 顧客
customers = ['C1', 'C2', 'C3', 'C4']

# 輸送費用 (F4を追加)
transport_costs = {
    ('F1', 'C1'): 20, ('F1', 'C2'): 10, ('F1', 'C3'): 50, ('F1', 'C4'): 30,
    ('F2', 'C1'): 60, ('F2', 'C2'): 30, ('F2', 'C3'): 20, ('F2', 'C4'): 10,
    ('F3', 'C1'): 10, ('F3', 'C2'): 40, ('F3', 'C3'): 15, ('F3', 'C4'): 50,
    ('F4', 'C1'): 18, ('F4', 'C2'): 12, ('F4', 'C3'): 20, ('F4', 'C4'): 10,
}

# --- モデル作成 ---
model = Model("FacilityLocation_Ex5_NewFacility")

# --- 決定変数 ---
y = {i: model.add_var(var_type=BINARY, name=f"y_{i}") for i in facilities}
x = {(i, j): model.add_var(var_type=BINARY, name=f"x_{i}_{j}") for i in facilities for j in customers}

# --- 目的関数 ---
model.objective = minimize(
    xsum(fixed_costs[i] * y[i] for i in facilities) +
    xsum(transport_costs[i, j] * x[i, j] for i in facilities for j in customers)
)

# --- 制約条件 ---
# 1. 各顧客は必ず1つの施設に割り当てられる
for j in customers:
    model += xsum(x[i, j] for i in facilities) == 1, f"AssignCust_{j}"

# 2. 顧客はその施設が開設されている場合にのみ割り当て可能
for i in facilities:
    for j in customers:
        model += x[i, j] <= y[i], f"OpenFac_{i}_{j}"

# --- モデルの最適化 ---
status = model.optimize()

# --- 結果の表示 ---
print("--- 演習問題5 結果 ---")
if status == OptimizationStatus.OPTIMAL:
    print(f"最適解が見つかりました。")
    print(f"最適な総コスト: {model.objective_value:.2f}")

    print("\n開設された施設:")
    for i in facilities:
        if y[i].x >= 0.99:
            print(f"- {i}")

    print("\n顧客の割り当て:")
    for j in customers:
        for i in facilities:
            if x[i, j].x >= 0.99:
                print(f"- 顧客 {j} -> 施設 {i} (輸送費用: {transport_costs[i, j]})")
else:
    print(f"最適化が停止しました。ステータス: {status}")

--- 演習問題5 結果 ---
最適解が見つかりました。
最適な総コスト: 150.00

開設された施設:
- F4

顧客の割り当て:
- 顧客 C1 -> 施設 F4 (輸送費用: 18)
- 顧客 C2 -> 施設 F4 (輸送費用: 12)
- 顧客 C3 -> 施設 F4 (輸送費用: 20)
- 顧客 C4 -> 施設 F4 (輸送費用: 10)
